### Current catalog

In [7]:
import pandas as pd
import pathlib
import seedir as sd

catalog_path = '../../data/previous-catalog/18strains.REL-1302-SV-GRCm38.sdp.tab'
catalog_data = pd.read_csv(catalog_path, sep='\t', low_memory=False)
catalog_data.head()

,CHROM,START,END,FORMAT,129P2,129S1,129S5,AJ,AKRJ,BALBcJ,...,CASTEiJ,CBAJ,DBA2J,FVBNJ,LPJ,NODShiLtJ,NZOHlLtJ,PWKPhJ,SPRETEiJ,WSBEiJ
0,X,3476745,3477456,POS:CL:BP:TY,X:3476761-3477345;DEL;RAW;DEL,X:3476745-3477456;DEL;RAW;DEL,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,X,3681483,3682211,POS:CL:BP:TY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,X:3681483-3682211;DEL;RAW;DEL
2,X,3858775,3858777,POS:CL:BP:TY,0,0,0,0,0,0,...,0,0,0,X:3858775-3858777;INS;REF;H6INS,0,0,0,0,0,0
3,X,4274734,4277734,POS:CL:BP:TY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X:4274734-4277734;DEL;RAW;DEL,0
4,X,4381696,4387658,POS:CL:BP:TY,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X:4381696-4387658;DEL;RAW;DEL,0


### Generate bed files per strain per type:

In [33]:
def bed_from_catalog(catalog_file, target_dir, target_strain, unified_file, naming_method):
    
    """
    bed_from_catalog(catalog_file, target_dir, target_strain, unified_file, naming_method)

    Generates BED Files from current SVs x Strain catalog.
    
    Parameters
    ----------
    catalog_file : str
        Path of the input catalog.
    target_dir : str
        Folder in which the BED files will be created.
    target_strain : str
        Strain (col names from catalog) for which the BED files will be created or "ALL".
    unified_file : bool
        If true, a single file is generated instead of one file per SV Type
    naming_method: int
        Naming for SV Types, currenly supports two options (1,2):
        1 => First 3 Chars, 2 => Any String until character "|" is found
        
    Returns
    -------
    list
        Merged list of SV Types found
    """
    
    data_dicts = {}
    strain_data = []
    all_types = []

    for strain in catalog_data.columns[4:]:
        if(strain != target_strain and target_strain!="all"):
            continue

        strain_data=catalog_data[catalog_data[strain]!="0"][strain]
        strain_dicts=data_dicts.get(strain, {})

        for entry in strain_data:
            entry_data = entry.split(";")
            raw_pos = entry_data[0].split(":")

            if naming_method == 1:
                sv_type = entry_data[1][:3]
            else:
                sv_type = entry_data[1].split("|")[0].split("(")[0]

            if unified_file:
                sv_type =  "ALL"

            chr1="chr"+raw_pos[0]
            start_end=raw_pos[1].split("-")
            start=start_end[0]
            end=start_end[1]
            length = int(end)-int(start)

            strain_type_dict = strain_dicts.get(sv_type, [])
            strain_type_dict.append({"chr": chr1, "start": start, "end": end, "len": str(length)}) 

            strain_dicts[sv_type]=strain_type_dict

        data_dicts[strain]=strain_dicts

        final_dir = target_dir + strain + "/"
        pathlib.Path(final_dir).mkdir(parents=True, exist_ok=True)

        for type_dict in data_dicts[strain].keys():
            type_df = pd.DataFrame(data_dicts[strain][type_dict])
            type_df.to_csv(final_dir + strain + "_" + type_dict + ".bed", sep='\t', index=False, header=False)
        
        all_types = list(set(all_types) | set([*data_dicts[strain]]))
        
    return all_types

In [34]:
target_dir="../../data/previous-catalog/"
target_strain="DBA2J"
unified_file=False
naming_method=2

bed_from_catalog(catalog_path, target_dir, "DBA2J", False, 2)

['TANDEMDUP',
 'INVDEL',
 'DELLINKED',
 'DELINS',
 'INSLINKEDINS',
 'DEL',
 'INVINS',
 'INS',
 'INVDELINS',
 'INV',
 'GAIN',
 'INSLINKED',
 'TANDEMLOWDUP',
 'TANDEMDUPINV']

### Generated files:

In [32]:
sd.seedir(target_dir, style='emoji', itemlimit=5, depthlimit=2, include_files='.*\.bed$', regex=True, sort=True)

📁 previous-catalog/
└─📁 DBA2J/
  ├─📄 DBA2J_DEL.bed
  ├─📄 DBA2J_DELINS.bed
  ├─📄 DBA2J_DELLINKED.bed
  ├─📄 DBA2J_GAIN.bed
  └─📄 DBA2J_INS.bed
